### Use this document to clean data for further use. Store cleaned dataset(s) in SQL to pull in other working files.

#### If too difficult with SQL, do with MongoDB and PyMongo

In [1]:
#import dependencies (Clean)
import numpy as np
import pandas as pd
from pprint import pprint

# import dependencies (Visuals - helpful for EDA & evaluation)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load CSV from a folder named 'Resources'
survey_df = pd.read_csv('../Resources/MentalHealthTechWorkplace2014.csv')
survey_df.head()


,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [3]:
# Look at our dataset from a distance
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   state                      744 non-null    object
 5   self_employed              1241 non-null   object
 6   family_history             1259 non-null   object
 7   treatment                  1259 non-null   object
 8   work_interfere             995 non-null    object
 9   no_employees               1259 non-null   object
 10  remote_work                1259 non-null   object
 11  tech_company               1259 non-null   object
 12  benefits                   1259 non-null   object
 13  care_options               1259 non-null   object
 14  wellness

Start cleaning/dropping nulls based on our previous EDA (file "RawExplore.ipynb")

In [4]:
# Check how many nulls
pprint(survey_df.isnull().sum())

Timestamp                       0
Age                             0
Gender                          0
Country                         0
state                         515
self_employed                  18
family_history                  0
treatment                       0
work_interfere                264
no_employees                    0
remote_work                     0
tech_company                    0
benefits                        0
care_options                    0
wellness_program                0
seek_help                       0
anonymity                       0
leave                           0
mental_health_consequence       0
phys_health_consequence         0
coworkers                       0
supervisor                      0
mental_health_interview         0
phys_health_interview           0
mental_vs_physical              0
obs_consequence                 0
comments                     1095
dtype: int64


### Clean Data I'll need for ML model

In [5]:
# Clean off odd age inputs (ex. 8 year-old, 300 year old, etc.)
print(f"Rows before removing outliers: {survey_df.shape[0]}")
survey_df_clean = survey_df[(survey_df['Age'] >= 18) & (survey_df['Age'] <= 100)]

# Check how many rows remain
print(f"Rows after removing outliers: {survey_df_clean.shape[0]}")
survey_df_clean.tail()

Rows before removing outliers: 1259
Rows after removing outliers: 1251


,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
1254,2015-09-12 11:17:21,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,46,f,United States,NC,No,No,No,NaN,100-500,...,Don't know,Yes,No,No,No,No,No,No,No,NaN
1258,2016-02-01 23:04:31,25,Male,United States,IL,No,Yes,Yes,Sometimes,26-100,...,Don't know,Maybe,No,Some of them,No,No,No,Don't know,No,NaN


In [6]:
# Bin ages to make it categorical (not continuous). want option to reduce noise for ML
# create bins
bins = [0, 20, 40, 60, 80, 100]
labels = ['<20', '20-39', '40-59', '60-79', '80+'] #keep in mind, bc dropped some ages, <20 is now basically 18 and 19 y/os

survey_df_clean['Age_binned'] = pd.cut(survey_df_clean['Age'], bins=bins, labels=labels, right=False)

# Optional: check distribution
print(survey_df_clean['Age_binned'].value_counts())

Age_binned
20-39    1054
40-59     175
<20        16
60-79       6
80+         0
Name: count, dtype: int64


C:\Users\musam\AppData\Local\Temp\ipykernel_14092\4254279781.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_clean['Age_binned'] = pd.cut(survey_df_clean['Age'], bins=bins, labels=labels, right=False)


In [7]:
# Drop all participants outside of US that input a US state (may be fine, but want to remove any potential error-ful subsmissions)
# Shape before changes
print(f"Shape before dropping non-US state errors: {survey_df_clean.shape}")

# Compound condition for dropping rows that meet this criteria
condition = (survey_df_clean['Country'] != 'United States') & survey_df_clean['state'].notnull()

# Drop rows where the condition is true
survey_df_clean.drop(survey_df_clean[condition].index, inplace=True)

print(f"Shape after dropping non-US state errors: {survey_df_clean.shape}")
print("----------------------")
survey_df_clean.head()

Shape before dropping non-US state errors: (1251, 28)
Shape after dropping non-US state errors: (1248, 28)
----------------------


C:\Users\musam\AppData\Local\Temp\ipykernel_14092\1527590974.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_clean.drop(survey_df_clean[condition].index, inplace=True)


,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments,Age_binned
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN,20-39
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Maybe,No,No,No,No,No,Don't know,No,NaN,40-59
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,No,No,Yes,Yes,Yes,Yes,No,No,NaN,20-39
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN,20-39
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN,20-39


In [8]:
# Create a binary column: "Yes" if comment exists, "No" if NaN
survey_df_clean['GaveComment'] = survey_df_clean['comments'].notnull().map({True: 'Yes', False: 'No'})
survey_df_clean.head()

C:\Users\musam\AppData\Local\Temp\ipykernel_14092\2735386843.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_clean['GaveComment'] = survey_df_clean['comments'].notnull().map({True: 'Yes', False: 'No'})


,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments,Age_binned,GaveComment
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,No,Some of them,Yes,No,Maybe,Yes,No,NaN,20-39,No
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,No,No,No,No,No,Don't know,No,NaN,40-59,No
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,No,Yes,Yes,Yes,Yes,No,No,NaN,20-39,No
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN,20-39,No
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN,20-39,No


In [9]:
# Clean up Gender column and create new categorical column

# 1. Find all unique text input for Gender
unique_genders = survey_df_clean['Gender'].unique()
# 2. Sort the unique values alphabetically to help me later
sorted_unique_genders = sorted(unique_genders)
print(sorted_unique_genders)

['Agender', 'Androgyne', 'Cis Female', 'Cis Male', 'Cis Man', 'Enby', 'F', 'Femake', 'Female', 'Female ', 'Female (cis)', 'Female (trans)', 'Genderqueer', 'Guy (-ish) ^_^', 'M', 'Mail', 'Make', 'Mal', 'Male', 'Male ', 'Male (CIS)', 'Male-ish', 'Malr', 'Man', 'Nah', 'Neuter', 'Trans woman', 'Trans-female', 'Woman', 'cis male', 'cis-female/femme', 'f', 'femail', 'female', 'fluid', 'm', 'maile', 'male', 'male leaning androgynous', 'msle', 'non-binary', 'ostensibly male, unsure what that really means', 'queer', 'queer/she/they', 'something kinda male?', 'woman']


In [10]:
# 3. Define mapping with dictionary (mapping method to re-categorize Gender)
gender_mapping = {
    'Cis Male': 'Man', 'Cis Man': 'Man', 'Guy (-ish) ^_^': 'Man', 'M': 'Man', 'Mail': 'Man', 'Make': 'Man', 'Mal': 'Man', 'Male': 'Man',\
    'Male ': 'Man', 'Male (CIS)': 'Man', 'Male-ish': 'Man', 'Malr': 'Man', 'Man': 'Man', 'cis male': 'Man', 'm': 'Man', 'maile': 'Man',\
    'male': 'Man', 'msle': 'Man', 'ostensibly male': 'Man', 'something kinda male?': 'Man',\
    
    'Cis Female': 'Woman', 'Femake': 'Woman', 'Female':'Woman', 'Female ':'Woman', 'Female (cis)':'Woman', 'Female (trans)':'Woman', \
    'Trans woman':'Woman', 'Trans-female':'Woman', 'Woman':'Woman', 'cis-female/femme': 'Woman', 'f': 'Woman', 'femail':'Woman', 
    'female':'Woman', 'woman': 'Woman', \
    
    'Agender': 'Other', 'Androgyne':'Other', 'Enby': 'Other', 'Genderqueer': 'Other', 'Nah': 'Other', 'Neuter': 'Other', \
    'gender non-conforming': 'Other', 'transgender': 'Other', 'fluid': 'Other', 'male leaning androgynous': 'Other', 'non-binary': 'Other',\
    'unsure what that really means': 'Other', 'queer': 'Other', 'queer/she/they': 'Other'
}

# 2. map onto df and bin to Other if not in dict
def map_gender(gender):
    for category, values in gender_mapping.items():
        # Check if gender matches any synonym in the list
        if gender in category:
            return values
    ## If no match found, return 'Other'
    else: return 'Other'

# 3. Apply the function to the Gender column
survey_df_clean['Gender_bin'] = survey_df_clean['Gender'].apply(map_gender)

#4. Check the updated Gender column
print(survey_df_clean['Gender_bin'].value_counts())
survey_df_clean.info()
survey_df_clean.head()

Gender_bin
Man      986
Woman    250
Other     12
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1248 entries, 0 to 1258
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Timestamp                  1248 non-null   object  
 1   Age                        1248 non-null   int64   
 2   Gender                     1248 non-null   object  
 3   Country                    1248 non-null   object  
 4   state                      735 non-null    object  
 5   self_employed              1230 non-null   object  
 6   family_history             1248 non-null   object  
 7   treatment                  1248 non-null   object  
 8   work_interfere             987 non-null    object  
 9   no_employees               1248 non-null   object  
 10  remote_work                1248 non-null   object  
 11  tech_company               1248 non-null   object  
 12  benefits           

C:\Users\musam\AppData\Local\Temp\ipykernel_14092\2731921844.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_clean['Gender_bin'] = survey_df_clean['Gender'].apply(map_gender)


,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments,Age_binned,GaveComment,Gender_bin
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Some of them,Yes,No,Maybe,Yes,No,NaN,20-39,No,Woman
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,No,No,No,No,Don't know,No,NaN,40-59,No,Man
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Yes,Yes,Yes,Yes,No,No,NaN,20-39,No,Man
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Some of them,No,Maybe,Maybe,No,Yes,NaN,20-39,No,Man
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Some of them,Yes,Yes,Yes,Don't know,No,NaN,20-39,No,Man


In [11]:
# ---will likely not use for this model BUT: ---
# Change timestamp to datetime data type so I can make Year and Month-Year columns for any future models/viz/fun
# NOTE: may need to change Month-Year to a different dtype in future for manipulation
survey_df_clean['Timestamp'] = pd.to_datetime(survey_df_clean['Timestamp'], errors='coerce')
# NOTE: when do .loc ver of this, for whatever reason doesn't convert to dt

# Add columns for years and month-years
survey_df_clean.loc[:, 'Year'] = survey_df_clean['Timestamp'].dt.year
survey_df_clean.loc[:, 'Month-Year'] = survey_df_clean['Timestamp'].dt.to_period('M')

pprint(survey_df_clean.dtypes)
print("--------------------")
survey_df_clean.head()

Timestamp                    datetime64[ns]
Age                                   int64
Gender                               object
Country                              object
state                                object
self_employed                        object
family_history                       object
treatment                            object
work_interfere                       object
no_employees                         object
remote_work                          object
tech_company                         object
benefits                             object
care_options                         object
wellness_program                     object
seek_help                            object
anonymity                            object
leave                                object
mental_health_consequence            object
phys_health_consequence              object
coworkers                            object
supervisor                           object
mental_health_interview         

C:\Users\musam\AppData\Local\Temp\ipykernel_14092\4109329417.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_clean['Timestamp'] = pd.to_datetime(survey_df_clean['Timestamp'], errors='coerce')
C:\Users\musam\AppData\Local\Temp\ipykernel_14092\4109329417.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_clean.loc[:, 'Year'] = survey_df_clean['Timestamp'].dt.year
C:\Users\musam\AppData\Local\Temp\ipykernel_14092\4109329417.py:9: SettingWithCopyWarning: 
A value is trying to be set on 

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments,Age_binned,GaveComment,Gender_bin,Year,Month-Year
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,No,Maybe,Yes,No,NaN,20-39,No,Woman,2014,2014-08
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,No,No,Don't know,No,NaN,40-59,No,Man,2014,2014-08
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Yes,Yes,No,No,NaN,20-39,No,Man,2014,2014-08
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Maybe,Maybe,No,Yes,NaN,20-39,No,Man,2014,2014-08
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Yes,Yes,Don't know,No,NaN,20-39,No,Man,2014,2014-08


### Make SQL-Python connection to store cleaned dataframe (survey_df_clean). This will make further ML work (04_Machine1.ipynb) and future tweaks much easier for us!

In [12]:
# import dependencies for SQL-Python connection
# set up SQL connection in python
# can do pyspark (module 22) OR psycopg2 (module 10)?

In [13]:
# # Python SQL toolkit and Object Relational Mapper
# import sqlalchemy
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func
# from sqlalchemy import Column, Integer, String, Float
# from sqlalchemy.ext.declarative import declarative_base #pick this or automap base?
# from sqlalchemy.ext.automap import automap_base #pick this or declaritive base? --> need to create class objects etc. easy enter instances of this
# # this class object into table as needed... will likely not need to do this for this project [SEE DECLARATIVE BASE EXAMPLE BELOW]
# Base = declarative_base() #because foundation SQL class object?? [SEE AUTOMAP BASE EXAMPLE]

In [14]:
# # Create Database Connection
# # database_path = "pets_db" CHANGE NAME to connect to the database I want to use
# engine = create_engine(f"sqlite:///{database_path}")
# conn = engine.connect()

In [15]:
# # [DECLARATIVE BASE EXAMPLE] if creating classes and object instances, see example below:
# # Create a Cat Class for the table. OUR BLUEPRINTS
# class Cat(Base):
#     __tablename__ = 'cat'
#     id = Column(Integer, primary_key=True)
#     name = Column(String(255))
#     color = Column(String(255))
#     age = Column(Integer)

# # Create a Dog Class for the table. 
# class Dog(Base):
#     __tablename__ = 'dog'
#     id = Column(Integer, primary_key=True)
#     name = Column(String(255))
#     color = Column(String(255))
#     age = Column(Integer)

# # Create a Specific Instance of the Dog and Cat classes
# dog = Dog(name="Fido", color='Brown', age=4) #create instance of Dog class
# cat = Cat(name="Whiskers", color="Gray", age=7)

# # Create a "Metadata" Layer That Abstracts our SQL Database 
# #data about data which will be dinfiition of columns in the ___?
# #tables created via create_all below
# Base.metadata.create_all(engine)


# # Create a Session Object to Connect to DB - session is talking to engine declared up there
# from sqlalchemy.orm import Session
# session = Session(bind=engine)

# # Add Records to the Appropriate DB
# session.add(dog) #create in CRUD; adding info for Fido etc, but gave a variable name bc working in python
# session.add(cat)
# session.commit()

# # Query the Dog table; if run multiple times, get multiple outputs unless clear this out
# dog_list = session.query(Dog) #querying dog table that got from class Dog
# for doggy in dog_list:
#     print(doggy.name)
#     #get Fido bc first info in name column
#     #can do doggy.color for fur color
## ALTERNATIVE EXAMPLE: MODULE 10 "surfer_SQL_solution" OR "ice_cream_connector_solution"

In [16]:
# # [AUTOMAP BASE] note: used sqlite file to start off with... how do without pre-made sqlite file? or first add uncleaned to postgreSQL then clean then
# # reload cleaned into new table in database?
# # create engine to hawaii.sqlite
# engine = create_engine("sqlite:///Resources/hawaii.sqlite")

**(doing this here due to lack of time, but move this work to file 04 at a later point once have SQL connection up and running)**
### Logistic Regression ML Model
Let's do some feature engineering for our model and then see if we can use it to predict treatment-seeking behaviors based off of our participants' entries.

In [17]:
# Scikit-learn tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [18]:
# Review of current columns
pprint(survey_df_clean.columns)

Index(['Timestamp', 'Age', 'Gender', 'Country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments', 'Age_binned',
       'GaveComment', 'Gender_bin', 'Year', 'Month-Year'],
      dtype='object')


In [19]:
# Make Copy of survey_df_clean to do feature work while preserving original
survey_df_model = survey_df_clean.copy()

# Drop all other columns I won't be using in ML
survey_df_model = survey_df_model.drop(columns=['Timestamp', 'Age', 'Gender', 'state', 'comments', 'Month-Year', 'Year']) 
survey_df_model.columns

Index(['Country', 'self_employed', 'family_history', 'treatment',
       'work_interfere', 'no_employees', 'remote_work', 'tech_company',
       'benefits', 'care_options', 'wellness_program', 'seek_help',
       'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'Age_binned', 'GaveComment',
       'Gender_bin'],
      dtype='object')

NOTE: seemingly duplicate columns (not adding extra info) not dropped with drop_first = True on original run:
- 'benefits_No', 'benefits_Yes'
- 'wellness_program_No', 'wellness_program_Yes'
-  'seek_help_No', 'seek_help_Yes'
-  'anonymity_No'  'anonymity_Ye
- 'mental_health_consequence_No', 'mental_health_consequence_Yes',
-  'phys_health_consequence_No', 'phys_health_consequence_Yes
- 'mental_health_interview_No', 'mental_health_interview_Yes'
-  'phys_health_interview_No'  'phys_health_interview_Yes
- , 'mental_vs_physical_No   'mental_vs_physical_Y

**will not manually make them binary with map method and remove from one-hot encode bulk session because of actually categorical (ex. Yes/No/Maybe).**

In [20]:
# Use this to quickly check unique responses for columns of interest
# print(survey_df_model['wellness_program'].nunique())
print(survey_df_model['mental_vs_physical'].nunique())
unique = survey_df_model['mental_vs_physical'].unique()
print(unique)

3
['Yes' "Don't know" 'No']


In [21]:
# One-Hot Encode all categorical string columns
#REMEMBER: may not want to do this for ordinal ex. Low, Medium, High...
survey_df_encoded = pd.get_dummies(survey_df_model, columns=['Country', 'self_employed', 'family_history', 'treatment',
                                                             'work_interfere', 'no_employees', 'remote_work', 'tech_company',
                                                             'benefits', 'care_options', 'wellness_program', 'seek_help',
                                                             'anonymity', 'leave', 'mental_health_consequence',
                                                             'phys_health_consequence', 'coworkers', 'supervisor',
                                                             'mental_health_interview', 'phys_health_interview',
                                                             'mental_vs_physical', 'obs_consequence', 'Age_binned', 'GaveComment',
                                                             'Gender_bin'], drop_first=True)
print(survey_df_encoded.columns)
print("--------------------------")
survey_df_encoded.head()

# Standard Scalar for numeric features so not overblown compared to encoded categorical features. Not applicable here :)

Index(['Country_Austria', 'Country_Belgium', 'Country_Bosnia and Herzegovina',
       'Country_Brazil', 'Country_Bulgaria', 'Country_Canada', 'Country_China',
       'Country_Colombia', 'Country_Costa Rica', 'Country_Croatia',
       'Country_Czech Republic', 'Country_Denmark', 'Country_Finland',
       'Country_France', 'Country_Georgia', 'Country_Germany',
       'Country_Greece', 'Country_Hungary', 'Country_India', 'Country_Ireland',
       'Country_Israel', 'Country_Italy', 'Country_Japan', 'Country_Mexico',
       'Country_Moldova', 'Country_Netherlands', 'Country_New Zealand',
       'Country_Nigeria', 'Country_Norway', 'Country_Philippines',
       'Country_Poland', 'Country_Portugal', 'Country_Romania',
       'Country_Russia', 'Country_Singapore', 'Country_Slovenia',
       'Country_South Africa', 'Country_Spain', 'Country_Sweden',
       'Country_Switzerland', 'Country_Thailand', 'Country_United Kingdom',
       'Country_United States', 'Country_Uruguay', 'self_employed_Yes',

,Country_Austria,Country_Belgium,Country_Bosnia and Herzegovina,Country_Brazil,Country_Bulgaria,Country_Canada,Country_China,Country_Colombia,Country_Costa Rica,Country_Croatia,...,mental_vs_physical_No,mental_vs_physical_Yes,obs_consequence_Yes,Age_binned_20-39,Age_binned_40-59,Age_binned_60-79,Age_binned_80+,GaveComment_Yes,Gender_bin_Other,Gender_bin_Woman
0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,False,True,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [22]:
# Define your features and target
X = survey_df_encoded.drop(columns=['treatment_Yes']) 
y = survey_df_encoded['treatment_Yes']

# Optional: confirm data types
print(X.dtypes)

Country_Austria                   bool
Country_Belgium                   bool
Country_Bosnia and Herzegovina    bool
Country_Brazil                    bool
Country_Bulgaria                  bool
                                  ... 
Age_binned_60-79                  bool
Age_binned_80+                    bool
GaveComment_Yes                   bool
Gender_bin_Other                  bool
Gender_bin_Woman                  bool
Length: 92, dtype: object


In [23]:
# Split data into train and test groups
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # stratify helps preserve class distribution
)

pprint(X_train.head())

      Country_Austria  Country_Belgium  Country_Bosnia and Herzegovina  \
383             False            False                           False   
68              False            False                           False   
1214            False            False                           False   
661             False            False                           False   
869             False            False                           False   

      Country_Brazil  Country_Bulgaria  Country_Canada  Country_China  \
383            False             False           False          False   
68             False             False           False          False   
1214           False             False           False          False   
661            False             False           False          False   
869            False             False           False          False   

      Country_Colombia  Country_Costa Rica  Country_Croatia  ...  \
383              False               False      

In [24]:
# Fit the Model
# Initialize the model
model = LogisticRegression(max_iter=1000)  # Increase max_iter if you get a convergence warning

# Fit the model to the training data
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [25]:
# Evaluate the Model
# Make predictions
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.844
Classification Report:
               precision    recall  f1-score   support

       False       0.86      0.81      0.84       124
        True       0.83      0.87      0.85       126

    accuracy                           0.84       250
   macro avg       0.85      0.84      0.84       250
weighted avg       0.85      0.84      0.84       250

Confusion Matrix:
 [[101  23]
 [ 16 110]]
